In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, time, datetime, timedelta
import matplotlib
from helpers import return_good_regions
import pickle

In [ ]:
base = '/Users/truggles/Downloads/for_tyler_Oct07_overimputes/'

df = pd.read_csv(base+'csv_MASTER_v12_2day.csv')
df['date_time'] = pd.to_datetime(df['date_time'])
df = df.drop(['SEC','SEC_category','OVEC','OVEC_category'], axis=1)

# Drop all *_category instances
regions = return_good_regions()
r_cats = [f'{r}_category' for r in regions]
df = df.drop(r_cats, axis=1)
df.head()

In [ ]:
# Normalize all columns
def normalize_each_column(df, cols):
    for col in cols:
        df[col] = df[col] / np.nanmean(df[col])
    return df

print(df['AECI'])
df = normalize_each_column(df, regions)
print(df['AECI'])

In [ ]:
def find_datetime_index(df, dt_target):
    
    if type(dt_target) == str:
        dt_target = datetime.strptime(dt_target, '%Y%m%dT%HZ')
    
    
    for idx in df.index:
        dt = df.loc[idx, 'date_time']
        #print(dt)
        if dt.year == dt_target.year and \
                dt.month == dt_target.month and \
                dt.day == dt_target.day and \
                dt.hour == dt_target.hour:
            print(f"{dt_target} at index {idx}")
            return idx, dt_target
    print(f"No datetime found in find_datetime_index for {dt_target}")

def plot_2D_range(df, cols, start_dt_str, length_dt):
    
    start_idx, start_dt = find_datetime_index(df, start_dt_str)
    end_dt = start_dt + timedelta(days=length_dt)
    end_idx, end_dt = find_datetime_index(df, end_dt)
    
    fig, ax = plt.subplots(figsize=(12,4))
    im = ax.imshow(df.loc[start_idx:end_idx, cols], interpolation='none', origin='lower', aspect='auto')

    # Axis ticks
    plt.xticks(np.arange(54), cols, rotation=90)
    dts = []
    for i in range(length_dt+1):
        dts.append(start_dt + timedelta(days=i))
    plt.yticks(np.arange(0, length_dt*24+1, 24), dts)

    cbar = ax.figure.colorbar(im)
    cbar.ax.set_ylabel("Normalized Demand")
    plt.tight_layout()
    plt.savefig("plot_2D_slice_{}_plus_{}_days.png".format(start_dt_str, length_dt))


plot_2D_range(df, regions, '20180115T00Z', 7)
plot_2D_range(df, regions, '20180415T00Z', 7)
plot_2D_range(df, regions, '20180715T00Z', 7)
plot_2D_range(df, regions, '20181015T00Z', 7)